In [ ]:
from time import time

from oemof import solph
import tsam.timeseriesaggregation as tsam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def preprocessing(noTypicalPeriods, hoursPerPeriod, data):
    aggregation = tsam.TimeSeriesAggregation(
        data,
        noTypicalPeriods=noTypicalPeriods,
        hoursPerPeriod=hoursPerPeriod,
        clusterMethod='k_means'
    )

    agg_data = aggregation.createTypicalPeriods()

    agg_data.index.names = ['TypicalPeriod', 'TimeStep']
    agg_data.reset_index(inplace=True)

    return aggregation, agg_data

In [ ]:
def optimization(agg_data):
    timeindex = pd.date_range('2019-01-01 00:00', freq='h', periods=agg_data.shape[0])
    es = solph.EnergySystem(
        timeindex=timeindex, infer_last_interval=True
    )

    b_gas = solph.Bus("gas bus")
    b_electricity = solph.Bus("electricity bus")
    b_heat = solph.Bus("heat bus")

    source_gas = solph.components.Source(
        "gas source",
        outputs={b_gas: solph.Flow(variable_costs=agg_data["gas_price"] + agg_data["co2_price"])}
    )
    source_electricity = solph.components.Source(
        "electricity source",
        outputs={b_electricity: solph.Flow(variable_costs=agg_data["el_spot_price"])}
    )
    sink_heat = solph.components.Sink(
        "heat demand",
        inputs={b_heat: solph.Flow(fix=agg_data["heat"], nominal_value=1)}
    )

    heat_pump = solph.components.Converter(
        label="heat pump",
        inputs={b_electricity: solph.Flow()},
        outputs={b_heat: solph.Flow(nominal_value=100)},
        conversion_factors={b_heat: 3.5}
    )
    boiler = solph.components.Converter(
        label="gas boiler",
        inputs={b_gas: solph.Flow()},
        outputs={b_heat: solph.Flow(nominal_value=100)},
        conversion_factors={b_heat: 0.9}
    )

    heat_slack = solph.components.Source(
        label="heat slack",
        outputs={b_heat: solph.Flow(variable_costs=1000)}
    )

    storage = solph.components.GenericStorage(
        label="heat storage",
        inputs={b_heat: solph.Flow(nominal_value=50)},
        outputs={b_heat: solph.Flow(nominal_value=50)},
        nominal_storage_capacity=24 * 50,
        initial_storage_level=0.5,
        balanced=True
    )

    es.add(
        b_gas, b_electricity, b_heat,
        source_electricity, source_gas, sink_heat, heat_slack,
        heat_pump, boiler, storage
    )

    model = solph.Model(es)

    _ = model.solve("gurobi")

    results = solph.views.convert_keys_to_strings(model.results())

    return results

In [ ]:
def postprocessing(results, agg_data=None, aggregation=None):
    unitdata = pd.DataFrame()
    for vertex, data in results.items():
        if vertex[-1] != 'None':
            unitdata[' to '.join(vertex)] = data['sequences']['flow']
        else:
            unitdata[f'{vertex[0]} storage content'] = data['sequences']['storage_content']

    if agg_data is not None and aggregation is not None:
        unitdata = unitdata.dropna()
        unitdata = unitdata.reset_index(drop=True)

        unitdata = pd.concat([unitdata, agg_data[['TypicalPeriod', 'TimeStep']]], axis=1)
        unitdata = unitdata.set_index(['TypicalPeriod', 'TimeStep'])

        matched_indices = aggregation.indexMatching()

        periods = unitdata.index.get_level_values('TypicalPeriod')
        timesteps = unitdata.index.get_level_values('TimeStep')

        unitdata_flat = pd.DataFrame(unitdata.values, columns=unitdata.columns)
        unitdata_flat['PeriodNum'] = periods
        unitdata_flat['TimeStep'] = timesteps

        desagg_data = matched_indices.reset_index().merge(
            unitdata_flat,
            how='left',
            left_on=['PeriodNum', 'TimeStep'],
            right_on=['PeriodNum', 'TimeStep']
        ).set_index(matched_indices.index)

        desagg_data = desagg_data.drop(columns=['Date', 'PeriodNum', 'TimeStep'])

        return desagg_data

    else:
        return unitdata.dropna()

In [ ]:
def fill_year(desagg_data, aggregation):
    matched_indices = aggregation.indexMatching()

    if desagg_data.index[0].is_leap_year:
        nr_missing = 8784 - len(desagg_data)
    else:
        nr_missing = 8760 - len(desagg_data)

    lastperiod = matched_indices[matched_indices['PeriodNum'] == matched_indices['PeriodNum'].iloc[-1]]
    lastperiod.drop_duplicates(inplace=True)
    lpdata = desagg_data.loc[lastperiod.iloc[:nr_missing].index , :]

    print(lpdata.shape)

    missing_index = pd.date_range(
        start=f'{desagg_data.index[-1]+pd.Timedelta("1h")}',
        end=f'{desagg_data.index[-1].year}-12-31 23:00',
        freq='h'
    )
    missing_data = pd.DataFrame(index=missing_index, data=lpdata.values, columns=lpdata.columns)

    return pd.concat([desagg_data, missing_data], axis=0)

In [ ]:
def calc_key_params(desagg_data, ts_in):
    el_cost = (desagg_data['electricity source to electricity bus'].values * ts_in['el_spot_price'].values).sum()
    gas_cost = (desagg_data['gas source to gas bus'].values * (
        ts_in['gas_price'].values + ts_in['co2_price'].values
    )).sum()
    slack_cost = (desagg_data['heat slack to heat bus'].values * 1000).sum()

    opex_total = el_cost + gas_cost  # + slack_cost

    heat_prod_shares = (
        desagg_data[['heat pump to heat bus', 'gas boiler to heat bus', 'heat slack to heat bus']].sum()
        / desagg_data[['heat pump to heat bus', 'gas boiler to heat bus', 'heat slack to heat bus']].sum().sum()
        )
    heat_prod_shares

    heat_prod_total = desagg_data['heat bus to heat demand'].sum()

    return opex_total, heat_prod_shares, heat_prod_total

In [ ]:
def test_agg_performance(noTypicalPeriods, hoursPerPeriod):
    ts_in = pd.read_csv("time_series.csv", index_col=0, parse_dates=True)
    ts_in = ts_in.drop(columns="ef_om")
    missing_hours = ts_in.shape[0] % hoursPerPeriod
    if missing_hours > 0:
        ts_in = ts_in.iloc[:-missing_hours, :]

    print('Perfomance Clustering & Preprocessing:')
    %timeit aggregation, agg_data = preprocessing(noTypicalPeriods=noTypicalPeriods, hoursPerPeriod=hoursPerPeriod, data=ts_in)
    print('Perfomance Optimization:')
    %timeit results = optimization(agg_data=agg_data)
    print('Perfomance Desaggregation:')
    %timeit desagg_data = postprocessing(results=results, agg_data=agg_data, aggregation=aggregation)

In [ ]:
def run_optimization_timed(noTypicalPeriods, hoursPerPeriod):
    ts_in = pd.read_csv("time_series.csv", index_col=0, parse_dates=True)
    ts_in = ts_in.drop(columns="ef_om")

    starttime = time()
    aggregation, agg_data = preprocessing(noTypicalPeriods=noTypicalPeriods, hoursPerPeriod=hoursPerPeriod, data=ts_in)
    aggregation_time = time() - starttime

    starttime = time()
    results = optimization(agg_data=agg_data)
    optimization_time = time() - starttime

    starttime = time()
    desagg_data = postprocessing(results=results, agg_data=agg_data, aggregation=aggregation)

    if desagg_data.index[0].is_leap_year:
        target_hours = 8784
        missing_hours = target_hours - len(desagg_data)
    else:
        target_hours = 8760
        missing_hours = target_hours - len(desagg_data)

    if missing_hours > 0:  # Is this case even occurring?
        desagg_data = fill_year(desagg_data=desagg_data, aggregation=aggregation)
    elif missing_hours < 0:
        desagg_data = desagg_data.iloc[:target_hours, :]

    disaggregation_time = time() - starttime

    opex_total, heat_prod_shares, heat_prod_total = calc_key_params(desagg_data=desagg_data, ts_in=ts_in)

    return aggregation_time, optimization_time, disaggregation_time, desagg_data, opex_total, heat_prod_shares, heat_prod_total, desagg_data

In [ ]:
ts_in = pd.read_csv("time_series.csv", index_col=0, parse_dates=True)
ts_in = ts_in.drop(columns="ef_om")

results = optimization(agg_data=ts_in)
unitdata_base = postprocessing(results)
opex_total_base, heat_prod_shares_base, heat_prod_total_base = calc_key_params(unitdata_base, ts_in)

heat_prod_shares_base

In [ ]:
opex_total_base / heat_prod_total_base

In [ ]:
heat_prod_total_base * 1000 * 0.0386

In [ ]:
noTypicalPeriods = 10
hoursPerPeriod = 168

ts_in = pd.read_csv("time_series.csv", index_col=0, parse_dates=True)
ts_in = ts_in.drop(columns="ef_om")
missing_hours = ts_in.shape[0] % hoursPerPeriod
if missing_hours > 0:
    ts_in = ts_in.iloc[:-missing_hours, :]
ts_in.shape

In [ ]:
aggregation, agg_data = preprocessing(noTypicalPeriods=noTypicalPeriods, hoursPerPeriod=hoursPerPeriod, data=ts_in)

In [ ]:
agg_data

In [ ]:
results = optimization(agg_data=agg_data)
results

In [ ]:
# desagg_data = postprocessing(results=results, agg_data=agg_data, aggregation=aggregation)
desagg_data = postprocessing(results=results, agg_data=agg_data, aggregation=aggregation)
desagg_data

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.plot(desagg_data['heat bus to heat demand'], label='Desaggregierte Wärmeproduktion')
ax.plot(ts_in['heat'], label='Gemessene Wärmeproduktion')

ax.legend()
ax.grid()
ax.set_ylabel('Stündliche Wärmeproduktion in MW')

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.plot(desagg_data['heat storage storage content'])

ax.grid()
ax.set_ylabel('Wärmespeicherfüllstand in MWh')

In [ ]:
aggdiff = desagg_data[['heat pump to heat bus', 'gas boiler to heat bus', 'heat slack to heat bus']] - unitdata_base[['heat pump to heat bus', 'gas boiler to heat bus', 'heat slack to heat bus']]
aggdiff.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.bar(list(range(1, len(desagg_data)+1)), desagg_data['heat pump to heat bus'], label='Einsatz desaggregiert')
ax.bar(list(range(1, len(aggdiff)+1)), aggdiff['heat pump to heat bus'], label='Abweichung Basecase')

ax.legend()
ax.grid(axis='y')
ax.set_ylabel('Abweichung Wärmeproduktion Wärmepumpe in MW')

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.bar(list(range(1, len(desagg_data)+1)), desagg_data['gas boiler to heat bus'], label='Einsatz desaggregiert')
ax.bar(list(range(1, len(aggdiff)+1)), aggdiff['gas boiler to heat bus'], label='Abweichung Basecase')

ax.legend()
ax.grid(axis='y')
ax.set_ylabel('Abweichung Wärmeproduktion Gaskessel in MW')

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.bar(list(range(1, len(desagg_data)+1)), desagg_data['heat slack to heat bus'], label='Einsatz desaggregiert')
ax.bar(list(range(1, len(aggdiff)+1)), aggdiff['heat slack to heat bus'], label='Abweichung Basecase')

ax.legend()
ax.grid(axis='y')
ax.set_ylabel('Abweichung Wärmeproduktion Wärmequelle in MW')

In [ ]:
test_agg_performance(noTypicalPeriods=10, hoursPerPeriod=168)

In [ ]:
import os

rerun = False
overwrite = False

nr_periods = list(range(1, 21))
hours_per_period = [24, 3*24, 5*24, 7*24, 14*24]

multiindex = pd.MultiIndex.from_product([hours_per_period, nr_periods])
perf = pd.DataFrame(index=multiindex)
keyparams = pd.DataFrame(index=multiindex)

heatflows = ['heat pump to heat bus', 'gas boiler to heat bus', 'heat slack to heat bus']

unit_data_all = {}
for hour_pp in hours_per_period:
    unit_data_all[hour_pp] = {}
    for nr_period in nr_periods:
        if rerun:
            aggregation_time, optimization_time, disaggregation_time, desagg_data, opex_total, heat_prod_shares, heat_prod_total, desagg_data = run_optimization_timed(noTypicalPeriods=nr_period, hoursPerPeriod=hour_pp)
            perf.loc[(hour_pp, nr_period), 'aggregation_time'] = aggregation_time
            perf.loc[(hour_pp, nr_period), 'optimization_time'] = optimization_time
            perf.loc[(hour_pp, nr_period), 'disaggregation_time'] = disaggregation_time
            keyparams.loc[(hour_pp, nr_period), 'opex_total'] = opex_total
            keyparams.loc[(hour_pp, nr_period), 'hp_share'] = heat_prod_shares['heat pump to heat bus']
            keyparams.loc[(hour_pp, nr_period), 'gb_share'] = heat_prod_shares['gas boiler to heat bus']
            keyparams.loc[(hour_pp, nr_period), 'slack_share'] = heat_prod_shares['heat slack to heat bus']
            keyparams.loc[(hour_pp, nr_period), 'heat_prod_total'] = heat_prod_total
            unit_data_all[hour_pp][nr_period] = desagg_data
            keyparams.loc[(hour_pp, nr_period), 'disp_diff_total'] = (desagg_data[heatflows] - unitdata_base[heatflows]).abs().sum().sum()
        else:
            path = os.path.join("results", f"{hour_pp}", f"{nr_period}", "result.csv")
            unit_data_all[hour_pp][nr_period] = pd.read_csv(path, sep=';', index_col=0, parse_dates=True)

        if overwrite:
            path = os.path.join("results", f"{hour_pp}", f"{nr_period}")
            os.makedirs(path, exist_ok=True)

            unit_data_all[hour_pp][nr_period].to_csv(os.path.join(path, "result.csv"), sep=";")

if rerun:
    perf['total_time'] = perf['aggregation_time'] + perf['optimization_time'] + perf['disaggregation_time']
else:
    keyparams = pd.read_csv(os.path.join("results", "keyparams.csv"), sep=";", index_col=[0, 1])
    perf = pd.read_csv(os.path.join("results", "perf.csv"), sep=";", index_col=[0, 1])

if overwrite:
    keyparams.to_csv(os.path.join("results", "keyparams.csv"), sep=";")
    perf.to_csv(os.path.join("results", "perf.csv"), sep=";")

In [ ]:
nr_axs = len(hours_per_period)
fig, ax = plt.subplots(nr_axs, 1, figsize=(12, 3*nr_axs))

for i, hour_pp in enumerate(hours_per_period):
    ax[i].bar(nr_periods, perf.loc[(hour_pp, nr_periods), 'aggregation_time'], label='aggregation')
    ax[i].bar(nr_periods, perf.loc[(hour_pp, nr_periods), 'optimization_time'], label='optimization', bottom=perf.loc[(hour_pp, nr_periods), 'aggregation_time'])
    ax[i].bar(nr_periods, perf.loc[(hour_pp, nr_periods), 'disaggregation_time'], label='disaggregation', bottom=perf.loc[(hour_pp, nr_periods), 'aggregation_time']+perf.loc[(hour_pp, nr_periods), 'optimization_time'])

    ax2 = ax[i].twinx()
    ax2.bar(
        nr_periods, keyparams.loc[(hour_pp, nr_periods), 'opex_total']
        /keyparams.loc[(hour_pp, nr_periods), 'heat_prod_total'],
        width=1/3, label='opex_total', color='red'
        )
    ax2.set_ylim(0, 1.05*(keyparams['opex_total']/keyparams['heat_prod_total']).max())
    ax2.set_ylabel('Operational cost per heat produce in €/MWh', color='red')
    ax2.axhline(opex_total_base/heat_prod_total_base, color='red')

    ax[i].grid()
    ax[i].set_ylim(0, 1.05*perf['total_time'].max())
    ax[i].set_xticks(nr_periods, labels=nr_periods)
    ax[i].set_title(f'{hour_pp} hours per period')
    ax[i].set_xlabel('Number of typical periods')
    ax[i].set_ylabel('Runtime in s')
    ax[i].legend(loc='upper left')

plt.tight_layout()

In [ ]:
nr_axs = len(hours_per_period)
fig, ax = plt.subplots(nr_axs, 1, figsize=(12, 3*nr_axs), sharey=True)

for i, hour_pp in enumerate(hours_per_period):
    _hp = ax[i].bar(nr_periods, keyparams.loc[(hour_pp, nr_periods), 'hp_share'], label='heat pump')
    _gb = ax[i].bar(nr_periods, keyparams.loc[(hour_pp, nr_periods), 'gb_share'], label='gas boiler', bottom=keyparams.loc[(hour_pp, nr_periods), 'hp_share'])
    _slack = ax[i].bar(nr_periods, keyparams.loc[(hour_pp, nr_periods), 'slack_share'], label='heat slack', bottom=keyparams.loc[(hour_pp, nr_periods), 'hp_share']+keyparams.loc[(hour_pp, nr_periods), 'gb_share'])
    ax[i].axhline(heat_prod_shares_base["heat pump to heat bus"], color=_hp[0]._facecolor)
    ax[i].axhline(heat_prod_shares_base["gas boiler to heat bus"] + heat_prod_shares_base["heat pump to heat bus"], color=_gb[0]._facecolor)
    ax[i].axhline(heat_prod_shares_base["heat slack to heat bus"] + heat_prod_shares_base["gas boiler to heat bus"] + heat_prod_shares_base["heat pump to heat bus"], color=_slack[0]._facecolor)

    ax[i].grid()
    ax[i].set_xticks(nr_periods, labels=nr_periods)
    ax[i].set_title(f'{hour_pp} hours per period')
    ax[i].set_xlabel('Number of typical periods')
    ax[i].set_ylabel('Share of heat production')
    ax[i].legend(loc='upper left')
    ax[i].set_axisbelow(True)

plt.tight_layout()

In [ ]:
nr_axs = len(hours_per_period)
fig, ax = plt.subplots(nr_axs, 1, figsize=(12, 3*nr_axs), sharey=True)

reference = heat_prod_shares_base.copy() * heat_prod_total_base
reference["lcoh"] = opex_total_base / heat_prod_total_base
# reference = reference.rename({"heat pump to heat bus": "hp_total", "gas boiler to heat bus": "gb_total", "heat slack to heat bus": "slack_total"})
reference = reference.rename({"heat pump to heat bus": "hp_total", "gas boiler to heat bus": "gb_total", "heat slack to heat bus": "slack_total"})
keyparams["lcoh"] = keyparams["opex_total"] / keyparams["heat_prod_total"]

val_min = -0.3
val_max = +0.3
values = np.linspace(val_min, val_max, 4)


cols = ["hp_share", "gb_share"]
for col in cols:
    keyparams[col.replace("share", "total")] = keyparams[col] * heat_prod_total_base


for i, hour_pp in enumerate(hours_per_period):
    for k, col in enumerate(['lcoh'] + [col.replace("share", "total") for col in cols]):
        value = keyparams.loc[(hour_pp, nr_periods), col]
        ax[i].bar(nr_periods + values[k], (value - reference[col]) / reference[col], label=col, width=0.2)


# for i, hour_pp in enumerate(hours_per_period):
#     for k, col in enumerate(['lcoh']):
#         value = keyparams.loc[(hour_pp, nr_periods), col]
#         ax[i].bar(nr_periods + values[k], (value - reference[col]), label=col, width=0.2)

# for i, hour_pp in enumerate(hours_per_period):
#     for k, col in enumerate(['hp_share', 'gb_share', 'slack_share']):
#         value = keyparams.loc[(hour_pp, nr_periods), col]
#         ax[i].bar(nr_periods + values[k], (value - reference[col]), label=col, width=0.2)

    ax[i].set_xticks(nr_periods, labels=nr_periods)
    ax[i].set_title(f'{hour_pp} hours per period')
    ax[i].set_ylabel('Absolute difference in heat shares to reference')
    ax[i].grid()
    ax[i].legend()
    ax[i].set_axisbelow(True)
    ax[i].set_ylim([-.03, +.03])

ax[-1].set_xlabel('Number of typical periods')
plt.tight_layout()

In [ ]:
import matplotlib as mpl

nr_axs = len(hours_per_period)
fig, ax = plt.subplots(nr_axs, 3, figsize=(12, 3*nr_axs))

for i, hour_pp in enumerate(hours_per_period):
    for nr_period in nr_periods:
        flows = ['heat pump to heat bus', 'gas boiler to heat bus', 'heat slack to heat bus']
        units = [f.replace(' to heat bus', '') for f in flows]
        unit_data_sorted = pd.DataFrame(
            np.sort(unit_data_all[hour_pp][nr_period][flows].values, axis=0)[::-1], columns=units
            )
        for u, unit in enumerate(units):
            ax[i][u].plot(unit_data_sorted[unit], c=mpl.cm.plasma(nr_period/max(nr_periods)))

    # heat pump

    for u, unit in enumerate(units):
        ax[i][u].grid()
        ax[i][u].set_title(f'{hour_pp} hours per period\n{unit}')
        ax[i][u].set_xlabel('Hour of year')
        ax[i][u].set_ylabel('Hourly heat production in MWh')

        ax[i][u].plot(unitdata_base[f"{unit} to heat bus"].sort_values(ascending=False).values)

fig.suptitle('Ordered yearly load duration curves by unit (warmer == more typical periods)\n')
plt.tight_layout()

In [ ]:
nr_axs = len(hours_per_period)
fig, ax = plt.subplots(nr_axs, 1, figsize=(12, 3*nr_axs), sharey=True)

flows = ['heat pump to heat bus', 'gas boiler to heat bus', 'heat slack to heat bus']
units = [f.replace(' to heat bus', '') for f in flows]
unitdata_base_sorted = pd.DataFrame(
    np.sort(unitdata_base[flows].values, axis=0)[::-1], columns=units
)
# unitdata_base_sorted
# unit_data_sorted

for i, hour_pp in enumerate(hours_per_period):
    per_period_df = pd.DataFrame(columns=units, index=nr_periods)
    for nr_period in nr_periods:
        unit_data_sorted = pd.DataFrame(
            np.sort(unit_data_all[hour_pp][nr_period][flows].values, axis=0)[::-1], columns=units
        )
        number_hours = len(unit_data_sorted)
        diff_rel = (unit_data_sorted - unitdata_base_sorted.iloc[:number_hours]).sum()

        per_period_df.loc[nr_period, units] = diff_rel

    locations = [-.25, 0, +.25]
    for k, unit in enumerate(units):
        ax[i].bar(per_period_df.index + locations[k], per_period_df[unit], width=0.25, label=unit)


    ax[i].set_xticks(nr_periods, labels=nr_periods)
    ax[i].set_title(f'{hour_pp} hours per period')
    ax[i].set_ylabel('Total difference in duration curve')
    ax[i].grid()
    ax[i].legend()
    ax[i].set_axisbelow(True)

    ax2 = ax[i].twinx()
    per_period_df.cumsum().plot.line(ax=ax2, legend=False)
    ax2.set_ylabel("Cumulative difference")
    ax2.set_ylim([-1, 2])

ax[-1].set_xlabel('Number of typical periods')
plt.tight_layout()


In [ ]:
# per_period_df.cumsum().plot(kind="line")

In [ ]:
nr_axs = len(hours_per_period)
fig, ax = plt.subplots(nr_axs, 1, figsize=(12, 3*nr_axs))

for i, hour_pp in enumerate(hours_per_period):
    ax[i].bar(nr_periods, keyparams.loc[(hour_pp, nr_periods), 'disp_diff_total'] / heat_prod_total_base)

    ax[i].grid(axis='y')
    # ax[i].set_ylim(0, 1.2)
    ax[i].set_xticks(nr_periods, labels=nr_periods)
    ax[i].set_title(f'{hour_pp} hours per period')
    ax[i].set_xlabel('Number of typical periods')
    ax[i].set_ylabel('Gesamte absolute Abweichung\ndes Anlageneinsatzes in MWh')

plt.tight_layout()

In [ ]:
heat_demand_total = unitdata_base["heat bus to heat demand"].sum()

In [ ]:
heat_demand_actual = {}
for hour_pp in unit_data_all:
    heat_demand_actual[hour_pp] = {}
    for nr_period, df in unit_data_all[hour_pp].items():
        heat_demand_actual[hour_pp][nr_period] = df["heat bus to heat demand"].sum()
        print(hour_pp, nr_period, (heat_demand_actual[hour_pp][nr_period] - heat_demand_total) / heat_demand_total)